In [ ]:
from glob import glob
import shutil
import glob
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from statistics import stdev
import networkx as nx
import pandas as pd
import codecs
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import pickle
from tqdm import tqdm
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
from sklearn.metrics import classification_report, precision_score,recall_score
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nit03/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data splitting for ai text detection

In [ ]:
def get_label(source):
    if source == 'human':
        return 0
    else:
        return 1

## reuter

In [ ]:
df = pd.read_csv('reuter_equal_length_intact_features.csv')
df.columns

Index(['key', 'source', 'intro_text', 'body_text', 'conclusion_text',
       'intro_word_count', 'body_word_count', 'conclusion_word_count',
       'total_word_count', 'intro_liwc_features', 'body_liwc_features',
       'conclusion_liwc_features'],
      dtype='object')

In [ ]:
df = df[['key','source','intro_text','body_text','conclusion_text']]
df.shape

(5401, 5)

In [ ]:
df = df.dropna()
df.shape

(5401, 5)

In [ ]:
df['text'] = df['intro_text'] + ' '+ df['body_text'] + ' ' + df['conclusion_text']
df['label'] = df['source'].apply(get_label)

In [ ]:
df.groupby('label').size()

label
0     872
1    4529
dtype: int64

In [ ]:
train_df , test_df = train_test_split(df,test_size=0.5)
valid_df , test_df = train_test_split(test_df, test_size=0.8)
print(train_df.shape, test_df.shape, valid_df.shape)

(2700, 7) (2161, 7) (540, 7)


In [ ]:
dataset = 'reuter'
train_df.to_csv(dataset+'//'+dataset+'_train.csv',index=False)
valid_df.to_csv(dataset+'//'+dataset+'_valid.csv',index=False)
test_df.to_csv(dataset+'//'+dataset+'_test.csv',index=False)

# Training the model

In [ ]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
import torch


In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device

device(type='mps')

In [ ]:
# Load tokenizer and model, move model to MPS if available
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## reuter

In [ ]:
dataset = 'reuter'
train_df = pd.read_csv(dataset+'//'+dataset+'_train.csv')
test_df = pd.read_csv(dataset+'//'+dataset+'_test.csv')
valid_df = pd.read_csv(dataset+'//'+dataset+'_valid.csv')
train_df = train_df.sample(frac=0.2)
valid_df = valid_df.sample(frac=0.2)
test_df = test_df.sample(frac=0.2)
train_df = train_df.reset_index()
valid_df = valid_df.reset_index()
test_df = test_df.reset_index()

In [ ]:
train_df.sample(2)

,index,key,source,intro_text,body_text,conclusion_text,text,label
536,746,KarlPenhaul_17,human,Another member of Colombia's main oil workers'...,USO head Hernando Hernandez said last Thursday...,Mines and Energy Minister Rodrigo Villamizar s...,Another member of Colombia's main oil workers'...,0
48,1569,JaneMacartney_6,llama2_chat,"Beijing, China - In a shocking turn of events,...",The sentence was handed down by the Beijing No...,The sentencing of Liu Xiaobo has sent shockwav...,"Beijing, China - In a shocking turn of events,...",1


In [ ]:
train_encodings = tokenizer(list(train_df['text']),truncation=True, padding="max_length",max_length=512)
valid_encodings = tokenizer(list(valid_df['text']), truncation=True, padding="max_length",max_length=512)
test_encodings = tokenizer(list(test_df['text']),truncation=True, padding="max_length",max_length=512)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_df['label'])
valid_dataset = CustomDataset(valid_encodings, valid_df['label'])
test_dataset = CustomDataset(test_encodings, test_df['label'])

In [ ]:
!pip install evaluate

In [ ]:
from transformers import Trainer, TrainingArguments
import evaluate  # Make sure to import this
def compute_metrics(eval_pred):
    f1_score = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_score.add_batch(predictions=predictions, references=labels)
    return f1_score.compute()

In [ ]:
model_output_dir = dataset + '_trained_models'

In [ ]:
from transformers import EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir=model_output_dir,  # output directory
    num_train_epochs=5,  # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=100,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    learning_rate=2e-5,  # learning rate
    save_total_limit=1,  # limit the total amount of checkpoints, delete older checkpoints
    logging_steps=100,
    evaluation_strategy="steps",  # evaluate at the end of each epoch
    save_strategy="steps",  # save model at the end of each epoch
    load_best_model_at_end=True,  # load the best model at the end of training
    metric_for_best_model="eval_loss",  # metric to track for early stopping
    greater_is_better=False,  # validation loss should decrease
)

# Create the Trainer object with Early Stopping Callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Early stopping patience
)

# Train the model
trainer.train()


  0%|          | 0/675 [00:00<?, ?it/s]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 2e-05, 'epoch': 0.74}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_f1': 0.0, 'eval_runtime': 6.0747, 'eval_samples_per_second': 17.779, 'eval_steps_per_second': 1.152, 'epoch': 0.74}
